In [1]:
%load_ext autoreload

In [2]:
import os
import sys
import re
import nltk
sys.path.append('../src-py/')
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [4]:
%autoreload
import prompted_conclusion_utils as conc_utils
from prompted_conclusion_utils import *
from ca_utils import *

cuda
2022-07-27 14:20:05,394 loading file ../../data-ceph/arguana/arg-generation/claim-target-tagger/model/final-model.pt
2022-07-27 14:20:33,961 SequenceTagger predicts: Dictionary with 4 tags: <unk>, B-CT, I-CT, O


In [5]:
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration

import torch

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

#data_unique_path = '/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl'
data_unique_path = '/reddit_data/conclusion_and_ca_generation/test_conclusion_all_preprocessed_multi_conclusions.pkl'
data_path = '/reddit_data/conclusion_and_ca_generation/test_conclusion_all.pkl'

In [8]:
def counters_coherence(post_conclusions, post_counters):
    post_counters = [nltk.sent_tokenize(x) for x in post_counters]
    conclusion_counter_sent_pairs = [(x[1], s) for x in zip(post_counters, post_conclusions) for s in x[0]]
    #print(conclusion_counter_sent_pairs)
    conclusions, counter_sents = zip(*conclusion_counter_sent_pairs)
    _, stance_labels, stance_scores = get_stance_scores(conclusions, counter_sents)
    stance_scores = [x[0] * -1 if x[1] == 0 else x[0] for x in zip(stance_scores, stance_labels)]

    #collect counter_scores
    counter_scores = []
    idx = 0
    #print(len(stance_scores))
    for i, post_counter in enumerate(post_counters):
        #print(len(post_counter))
        counter_scores.append(stance_scores[idx: idx + len(post_counter)])
        idx+=len(post_counter)
    
    #print(counter_scores)
    return [np.mean(s) for s in counter_scores]
        
def get_best_counter(post_item):
    if len(post_item) == 0:
        print('found empty')
        return ''
    if len(post_item) == 1:
        print('found one item')
        return post_item[0][2]
    
    # Remove empty counters
    post_item = [x for x in post_item if x[-1] != '']
    
    _, post_conclusions, post_counters = zip(*post_item)
    counter_scores = counters_coherence(post_conclusions, post_counters)
    #print(sorted(list(zip(counter_scores, post_counters)), key=lambda x: -x[0]))
    return post_counters[np.argmax(counter_scores)]

--------

### Generate multiple-counters (joint way):

In [9]:
gen_kwargs = {
    "do_sample": False, 
    "max_length":100+50,  #50 for the conclusion
    "min_length":50+50, # 50 for the conclusion
    "top_p":0.95, 
    "no_repeat_ngram_size":3,
    "top_k": 50,
    "num_beams":4
}

In [10]:
pred_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/ca-final-models/pred-conc-model').to(device)
pred_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/ca-final-models/pred-conc-model')

In [12]:
#Ideally we will start from sample_test_conclusion_all_preprocessed.pkl and generate the concepts
#But here we are just using the concepts generated in the pipeline way..
#test_df = pd.read_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed.pkl')
#test_df['post_concepts'] = conc_utils.get_wiki_concepts(test_df['post'].tolist(), max_concepts=10)

#test_df = pd.read_pickle('../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl')
test_df = pd.read_pickle('../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/test_conclusion_all_preprocessed_multi_conclusions.pkl')

In [13]:
def generate_multiple_counters(df, output_clm='post_counters', post_concepts_clm='post_concepts', num_counters=6, batch_size=8):
    posts_counters = []
    for i in range(num_counters):
        control_codes = ['<s> <conclusion>' + x[i] if len(x) > i else '' for x in df[post_concepts_clm].tolist()]
        #print(control_codes)
        counters   = generate_prompted_counters(pred_conclusion_model, pred_conclusion_tokenizer, 
                                                              df['post'].tolist(), control_codes, gen_kwargs, 
                                                              batch_size=batch_size)
        
        cases_with_no_counter = [x for x in counters if '<counter>' not in x]
        if len(cases_with_no_counter) > 0:
            print('{} cases with no counter'.format(len(cases_with_no_counter)))
            
        conclusions = [c.split('<counter>')[0] for c in counters]
        conclusions = [re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', c).strip() for c in conclusions] #removing special tokens
        
        counters    = [c.split('<counter>')[1] if '<counter>' in c else c for c in counters]
        counters    = [re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', c).strip() for c in counters] #removing special tokens
        
        control_codes = [x.replace('<s> <conclusion>', '') for x in control_codes]
        
        posts_counters.append(zip(control_codes, conclusions, counters))

    df['post_counters'] = list(map(list, zip(*posts_counters))) # transpose

    return df

In [14]:
#test_df_sample = generate_multiple_counters(test_df.sample(5), num_counters=2, batch_size=16)

In [ ]:
test_df = generate_multiple_counters(test_df, num_counters=10, batch_size=16)

In [ ]:
#test_df.to_pickle('../data/output/multi_conclusions_via_joint_model.pkl')
test_df.to_pickle('../data/output/test_all_multi_conclusions_via_joint_model_nosampling_4beams.pkl')

#### Generate with nuclus sampling:

In [15]:
gen_kwargs = {
    "do_sample": True, 
    "max_length":100+20,  #50 for the conclusion
    "min_length":50+20, # 50 for the conclusion
    "top_p":0.95, 
    "no_repeat_ngram_size":3,
    "top_k": 50,
    "num_beams":4
}

test_df = generate_multiple_counters(test_df, post_concepts_clm='post_concepts2', num_counters=10, batch_size=16)

Parameter 'function'=<function generate_prompted_counters.<locals>.<lambda> at 0x7fb12f7d69d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/9 [00:00<?, ?ba/s]

100%|██████████| 534/534 [51:57<00:00,  5.84s/it]


130 cases with no counter


  0%|          | 0/9 [00:00<?, ?ba/s]

...

  0%|          | 0/9 [00:00<?, ?ba/s]

100%|██████████| 534/534 [46:35<00:00,  5.23s/it]


104 cases with no counter


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 2/534 [00:10<44:37,  5.03s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [16]:
#test_df.to_pickle('../data/output/multi_conclusions_via_joint_model_withsampling_4beams.pkl')
test_df.to_pickle('../data/output/test_all_multi_conclusions_via_joint_model_withsampling_4beams.pkl')

In [17]:
test_df[test_df.post_counters.str.len() == 10][['title', 'post_counters']].head(n=10)

title  \
410850                                                                  people should come with instructions   
410858       People should not be heavily criticized for things they put on social media in the distant past   
410902                                                          We shouldn't focus on slowing climate change   
410910                   The Australian PM was right to tell students to stop activism around global warming   
410916                                                Feeding cats or dogs a diet with meat is indefensible.   
410927   For a government to allow Hate Speech on the basis of Free Speech they must also abolish Copyright.   
410940                             Cmv: if you are against legal abortion then you are for illegal abortion.   
410964                                                       The job market in the US has never been better.   
410971                      Men and women should not be treated the same when it comes to sexual harrassment   
410991                     All couples must undergo standarized tests in order to conceive and have children   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

#### Choose the best counter:

In [20]:
test_df = pd.read_pickle('../data/output/multi_conclusions_via_joint_model_withsampling_4beams.pkl')
#Using our stance classifier
test_df['best_counter'] = test_df.post_counters.apply(lambda post_item: get_best_counter(post_item))
test_df.to_pickle('../data/output/multi_conclusions_via_joint_model_withsampling_4beams.pkl')

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [28]:
test_df = pd.read_pickle('../data/output/multi_conclusions_via_joint_model_nosampling_4beams.pkl')
#Using our stance classifier
test_df['best_counter'] = test_df.post_counters.apply(lambda post_item: get_best_counter(post_item))
test_df.to_pickle('../data/output/multi_conclusions_via_joint_model_nosampling_4beams.pkl')

/usr/local/lib/python3.6/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [21]:
print('done')

done


-------

#### Testing:

In [32]:
test_df = pd.read_pickle('../data/output/multi_conclusions_via_joint_model_withsampling_4beams.pkl')

In [10]:
test_df[['title', 'best_counter']].head(n=20)

,title,best_counter
500025,I Don't Believe in the Big Bang,science is not the only source of truth. religion is. science is the only means by which we can know the truth. there is no other way to understand the universe other than through science.
534021,There is no reason for Britain to remain in the EU.,"the eu is not a single entity, it is a collection of countries. britain is not an island nation, but it is part of the eu. it is not british, but is a member of the european union."
415645,The Trump Administration Would Be Significantly More Harmful to US Domestic and International Policy W/out Moderates Like Mattis,"i think mattis has done a lot of good for the united states and the united nations, but i don't think he has done much good for americans or the world as a whole. he is not the most intelligent person in the world, and he doesn't have the best grasp of the world around him."
507298,Poisoning the well against fascists is a dishonest and therefore ineffective way of combating fascists,"i don't think it's disingenuous to say that 'white supremacy' isn't a fascism ideology. it's a white supremacist ideology, but it doesn't mean that all white supremacists are white supremacists. it just means that they're white nationalists."
505763,Women have an easier time finding romantic partners than men,"i don't think it's as easy as you make it out to be. it's not that women have a harder time finding romantic partners, it's that women are more selective about who they date. i don't know how to change your view, but it sounds like you're not happy with your situation."
432241,"White people should not be berated for their past in colonialism, slavery, and genocide.","i think it's fair to say that white people are responsible for a lot of these things, but that doesn't mean that they are the only ones responsible for them. i don't think that it's reasonable to blame white people for everything that happened in america. it's not fair to blame black people for anything that happened to them."
439718,"Free market Capitalism (Anarcho Capitalism, Laissez-Faire capitalism etc.) works the best, in opposition to Social-Democracy/Socialism, Communism.","the problem with the free market is that it doesn't work for all people. it's good for the rich, but it's bad for the poor, and it's not good for everyone. for example, it's great for the wealthy, but bad for poor people."
529099,"Nuclear power in the current state is not a alternative to solar, water, wind and other renewable sources.","nuclear power is a very efficient way of generating power. it is cheaper than coal, oil, and other fossil fuels, and it is much safer than coal and oil."
531796,If Conflict theory is correct then White people and Asian people need to start fighting against diversity rhetoric,"affirmative action is about equality, not about power. it is not about race. affirmative action isn't about power, it is about equal opportunity."
412902,It makes no sense (in secular occasions) to obsess over family themed holidays—such Thanksgiving and Christmas—when you have every other day of the year to be with family.,"family oriented holidays aren't meaningless because if you truly love your family, you should be able to meet up and spend time with them whenever you get a chance. they're not meaningless because there is no such thing as a'meaningless' holiday."


In [11]:
post_item = test_df.post_counters.tolist()[1]

In [12]:
post_concepts, post_conclusions, post_counters = zip(*post_item)
counters_coherence(post_conclusions, post_counters)

[0.8689321130514145,
 0.9768721659978231,
 0.6022831439971924,
 0.9013810157775879,
 0.9613067805767059,
 0.971846342086792,
 0.8979455629984537,
 0.955413031578064,
 0.944834957520167,
 0.907984983921051]

In [13]:
post_counters

("the eu laws are not 'bad' because they are good, they are bad because the eu doesn't follow the rules of the world. the laws are good because they allow free movement of goods and people. they aren't bad because they don't respect the rights of other countries. they are stupid because they do not respect the wishes of the people.",
 'the eu is not a single entity, it is a collection of countries. britain is not an island nation, but it is part of the eu. it is not british, but is a member of the european union.',
 'the eu is a trade bloc, not a federation of sovereign states. britain is an island nation, but is not a part of the eu. it is a trading bloc, but not a union of states. it does not have the same laws as the us, nor does it have the customs union with the us. the us is a member of the european union, but does not',
 "the eu is not a single entity. it is a collection of many smaller countries, all of which have their own cultures, traditions, languages, customs, etc. that ha

In [14]:
post_conclusions

('European Union law is bad and Britain should leave.',
 'Island country Britain should leave the European Union',
 'Trade blocs are bad and Britain should leave the EU.',
 'Britain should leave the EU',
 'Britain should leave the EU',
 'I think Britain should leave the EU.',
 'I think Britain should leave the EU',
 'Britain should leave the EU',
 'Britain should leave the EU.',
 'I think Britain should leave the EU.')

In [15]:
post_concepts

('European Union law',
 'Island country',
 'Trade bloc',
 '',
 '',
 '',
 '',
 '',
 '',
 '')